## FEATURE ENGINEERING

In [1]:
# import libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split


# text processing libraries
import re
import contractions

from collections import Counter
# import string
import nltk
# import warnings
# %matplotlib inline
# warnings.filterwarnings("ignore")
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud

from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\macie\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\macie\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [133]:
x_train = pd.read_csv("..//data//x_train.csv", encoding="latin-1")
y_train = pd.read_csv("..//data//y_train.csv", encoding="latin-1")
x_test = pd.read_csv("..//data//x_test.csv", encoding="latin-1")
y_test = pd.read_csv("..//data//y_test.csv", encoding="latin-1")
x_valid = pd.read_csv("..//data//x_valid.csv", encoding="latin-1")
y_valid = pd.read_csv("..//data//y_valid.csv", encoding="latin-1")

In [134]:
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

((513801, 5), (513801, 1), (220201, 5), (220201, 1))

In [81]:
x_train = x_train.drop(['ID', 'Date', 'flag', 'User'], axis = 'columns')
# removing unnecessary user tags
x_train['Text'] = x_train['Text'].replace(r"@\w+", "", regex=True)
# resolving contractions (and slang)
x_train['Text'] = x_train['Text'].apply(lambda x: contractions.fix(x))
# removing punctuation marks
x_train['Text'] = x_train['Text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
# deleting websites
x_train['Text'] = x_train['Text'].apply(lambda x: re.sub(r'http\S+', '', x))
# lowercasing letters in the text
x_train['Text'] = x_train['Text'].str.lower()
#x_train['Text'] = x_train['Text'].apply(lambda x: " ".join([w for w in x.split() if len(w) >= 2]))
# individual words considered as tokens
tokenized_tweet = x_train['Text'].apply(lambda x: x.split())
# Initialize wordnet lemmatizer only on verbs - makes the biggest sense
wnl = WordNetLemmatizer()
tokenized_tweet = tokenized_tweet.apply(lambda s: [wnl.lemmatize(word, pos='v') for word in s])
tokenized_tweet = tokenized_tweet.apply(lambda s: [wnl.lemmatize(word, pos='n') for word in s])
tokenized_tweet = tokenized_tweet.apply(lambda s: [wnl.lemmatize(word, pos='a') for word in s])
tokenized_tweet = tokenized_tweet.apply(lambda s: [wnl.lemmatize(word, pos='r') for word in s])
# combining to sentences
combined_sentences = [' '.join(tokens) for tokens in tokenized_tweet]
x_train['combined_tweet'] = combined_sentences
x_train


,Text,combined_tweet
810677,working listening to kmps happy my boss amp th...,work listen to kmps happy my bos amp the mecha...
684982,we both know she is late for quotat 11quot,we both know she be late for quotat 11quot
954730,do not bogart that joint my friend,do not bogart that joint my friend
142760,i am so sick of being sick i do not want to mi...,i be so sick of be sick i do not want to miss ...
200331,i am pretty sure i went to bed about 2 hours a...,i be pretty sure i go to bed about 2 hour ago ...
...,...,...
1023596,good morning sunshine,good morning sunshine
281932,is the website down i cannot get it to load,be the website down i cannot get it to load
828817,i listened to the itunes samplesmy fav albums...,i listen to the itunes samplesmy fav album be ...
683555,i use palringo for chat and fring for voice c...,i use palringo for chat and fring for voice ca...


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [82]:
custom_stop_words = CountVectorizer(stop_words='english').get_stop_words()
custom_stop_words = set(custom_stop_words) - {'not','alone','why','well','very','together','such','nobody','noone','nothing','myself','cry','cannot','cant','can','could','couldnt'}
custom_stop_words

{'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amoungst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'bill',
 'both',
 'bottom',
 'but',
 'by',
 'call',
 'co',
 'con',
 'de',
 'describe',
 'detail',
 'do',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eg',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'etc',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fifty',
 'fill',
 'find',
 'fire',
 'first',
 'five',
 'for',
 'former',
 'formerly',
 'forty',
 'found',
 'four',
 'from',
 'fr

In [127]:
bow_vectorizer = CountVectorizer(max_df = 0.95, min_df = 5, max_features = 5000, stop_words=custom_stop_words)
bow = bow_vectorizer.fit_transform(x_train['combined_tweet'])

In [84]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score

In [128]:
#training
model = LogisticRegression(max_iter=5000)
model.fit(bow, y_train)

LogisticRegression(max_iter=5000)

In [86]:
x_valid = x_valid.drop(['ID', 'Date', 'flag', 'User'], axis = 'columns')
# removing unnecessary user tags
x_valid['Text'] = x_valid['Text'].replace(r"@\w+", "", regex=True)
# resolving contractions (and slang)
x_valid['Text'] = x_valid['Text'].apply(lambda x: contractions.fix(x))
# removing punctuation marks
x_valid['Text'] = x_valid['Text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
# deleting websites
x_valid['Text'] = x_valid['Text'].apply(lambda x: re.sub(r'http\S+', '', x))
# lowercasing letters in the text
x_valid['Text'] = x_valid['Text'].str.lower()
#x_valid['Text'] = x_valid['Text'].apply(lambda x: " ".join([w for w in x.split() if len(w) >= 2]))
# individual words considered as tokens
tokenized_tweet = x_valid['Text'].apply(lambda x: x.split())
# Initialize wordnet lemmatizer only on verbs - makes the biggest sense
wnl = WordNetLemmatizer()
tokenized_tweet = tokenized_tweet.apply(lambda s: [wnl.lemmatize(word, pos='v') for word in s])
tokenized_tweet = tokenized_tweet.apply(lambda s: [wnl.lemmatize(word, pos='n') for word in s])
tokenized_tweet = tokenized_tweet.apply(lambda s: [wnl.lemmatize(word, pos='a') for word in s])
tokenized_tweet = tokenized_tweet.apply(lambda s: [wnl.lemmatize(word, pos='r') for word in s])
# combining to sentences
combined_sentences = [' '.join(tokens) for tokens in tokenized_tweet]
x_valid['combined_tweet'] = combined_sentences
x_valid

,Text,combined_tweet
240689,tierd and it is school tomorrow last week atl...,tierd and it be school tomorrow last week atleast
413003,twitter gets boring n boring everydayno star w...,twitter get bore n bore everydayno star want 2...
950284,i am watching guy ripley right nowhahahilarious,i be watch guy ripley right nowhahahilarious
672298,that is the way indoor stadium toilets are,that be the way indoor stadium toilet be
852721,it must be all that bike riding,it must be all that bike rid
...,...,...
55759,wantd 2b comedian when lil boy i memrize comm...,wantd 2b comedian when lil boy i memrize comme...
175608,omg i cannot believe jay leno is going off the...,omg i cannot believe jay leno be go off the air
661283,i do not know my days are all messed up since...,i do not know my day be all mess up since i ge...
43369,so i am guessin meant midnight pacific time,so i be guessin mean midnight pacific time


In [129]:
# bow_vectorizer = CountVectorizer(max_df=0.99, min_df=10, max_features=1000, stop_words=custom_stop_words)
bow = bow_vectorizer.transform(x_valid['combined_tweet'])

In [130]:
#testing
pred = model.predict(bow)
pred
sum(pred)

132324

In [131]:
from sklearn.metrics import f1_score
f1_score(y_valid, pred, pos_label=4)

0.5535216219383569

In [132]:
accuracy_score(y_valid,pred)

0.8269853452073334